In [88]:
import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib as plt 
import  csv
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from model_code import run_model

from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report

from sklearn import svm
from sklearn import tree
from sklearn import metrics


from sklearn.metrics import accuracy_score

In [89]:
df = pd.read_csv('TravelInsurancePrediction.csv')
df.drop(df.columns[[0]], axis=1, inplace=True)
df.head(10)

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,31,Government Sector,Yes,400000,6,1,No,No,0
1,31,Private Sector/Self Employed,Yes,1250000,7,0,No,No,0
2,34,Private Sector/Self Employed,Yes,500000,4,1,No,No,1
3,28,Private Sector/Self Employed,Yes,700000,3,1,No,No,0
4,28,Private Sector/Self Employed,Yes,700000,8,1,Yes,No,0
5,25,Private Sector/Self Employed,No,1150000,4,0,No,No,0
6,31,Government Sector,Yes,1300000,4,0,No,No,0
7,31,Private Sector/Self Employed,Yes,1350000,3,0,Yes,Yes,1
8,28,Private Sector/Self Employed,Yes,1450000,6,1,Yes,Yes,1
9,33,Government Sector,Yes,800000,3,0,Yes,No,0


In [97]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,2,6,7])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(X_train)
print(X_test)

[[ 1.55473609 -1.55473609 -0.41452251 ... -0.05365604  1.38182856
   1.6083866 ]
 [ 1.55473609 -1.55473609 -0.41452251 ...  0.7456513  -1.1034296
  -0.62174107]
 [ 1.55473609 -1.55473609 -0.41452251 ...  0.47921552 -1.1034296
  -0.62174107]
 ...
 [-0.64319598  0.64319598 -0.41452251 ... -1.11939916  0.13919948
   1.6083866 ]
 [-0.64319598  0.64319598 -0.41452251 ...  0.7456513  -1.1034296
  -0.62174107]
 [-0.64319598  0.64319598  2.41241424 ...  0.61243341 -0.48211506
  -0.62174107]]
[[-0.64319598  0.64319598 -0.41452251 ...  1.54495864  0.13919948
   1.6083866 ]
 [ 1.55473609 -1.55473609 -0.41452251 ...  0.7456513   1.38182856
   1.6083866 ]
 [-0.64319598  0.64319598 -0.41452251 ... -0.05365604 -1.1034296
  -0.62174107]
 ...
 [-0.64319598  0.64319598 -0.41452251 ...  1.27852286 -1.72474414
   1.6083866 ]
 [ 1.55473609 -1.55473609  2.41241424 ... -1.65227073  0.76051402
  -0.62174107]
 [ 1.55473609 -1.55473609 -0.41452251 ... -0.05365604  0.76051402
   1.6083866 ]]


Baseline models 

In [100]:
classifier_lr = LogisticRegression(random_state = 0)
classifier_nb= GaussianNB()
classifier_dt= DecisionTreeClassifier(random_state=0)
classifier_knn= KNeighborsClassifier(n_neighbors=3)

In [96]:
def run_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)    
    cnf_matrix_test = confusion_matrix(y_test,  y_pred_test)

    print('Confusion Matrix for Test Set:\n', cnf_matrix_test)
   
    cnf_matrix_train = confusion_matrix(y_train,  y_pred_train)
    print('Confusion Matrix for Train Set:\n', cnf_matrix_train)
 
    print('Classification Report for Test Set:\n',classification_report(y_test , y_pred_test))
    print('Classification Report for Train Set:\n',classification_report(y_train, y_pred_train))
    return model 


In [82]:
run_model(classifier_lr,X_train, X_test, y_train, y_test)

Confusion Matrix for Test Set:
 [[234  26]
 [ 67  71]]
Confusion Matrix for Train Set:
 [[941  76]
 [296 276]]
Classification Report for Test Set:
               precision    recall  f1-score   support

           0       0.78      0.90      0.83       260
           1       0.73      0.51      0.60       138

    accuracy                           0.77       398
   macro avg       0.75      0.71      0.72       398
weighted avg       0.76      0.77      0.75       398

Classification Report for Train Set:
               precision    recall  f1-score   support

           0       0.76      0.93      0.83      1017
           1       0.78      0.48      0.60       572

    accuracy                           0.77      1589
   macro avg       0.77      0.70      0.72      1589
weighted avg       0.77      0.77      0.75      1589



LogisticRegression(random_state=0)

In [83]:
run_model(classifier_nb,X_train, X_test, y_train, y_test)

Confusion Matrix for Test Set:
 [[206  54]
 [ 59  79]]
Confusion Matrix for Train Set:
 [[859 158]
 [273 299]]
Classification Report for Test Set:
               precision    recall  f1-score   support

           0       0.78      0.79      0.78       260
           1       0.59      0.57      0.58       138

    accuracy                           0.72       398
   macro avg       0.69      0.68      0.68       398
weighted avg       0.71      0.72      0.71       398

Classification Report for Train Set:
               precision    recall  f1-score   support

           0       0.76      0.84      0.80      1017
           1       0.65      0.52      0.58       572

    accuracy                           0.73      1589
   macro avg       0.71      0.68      0.69      1589
weighted avg       0.72      0.73      0.72      1589



GaussianNB()

In [84]:
run_model(classifier_dt,X_train, X_test, y_train, y_test)

Confusion Matrix for Test Set:
 [[217  43]
 [ 51  87]]
Confusion Matrix for Train Set:
 [[1003   14]
 [ 103  469]]
Classification Report for Test Set:
               precision    recall  f1-score   support

           0       0.81      0.83      0.82       260
           1       0.67      0.63      0.65       138

    accuracy                           0.76       398
   macro avg       0.74      0.73      0.74       398
weighted avg       0.76      0.76      0.76       398

Classification Report for Train Set:
               precision    recall  f1-score   support

           0       0.91      0.99      0.94      1017
           1       0.97      0.82      0.89       572

    accuracy                           0.93      1589
   macro avg       0.94      0.90      0.92      1589
weighted avg       0.93      0.93      0.92      1589



DecisionTreeClassifier(random_state=0)

In [101]:
run_model(classifier_knn,X_train, X_test, y_train, y_test)

Confusion Matrix for Test Set:
 [[229  31]
 [ 54  84]]
Confusion Matrix for Train Set:
 [[953  64]
 [163 409]]
Classification Report for Test Set:
               precision    recall  f1-score   support

           0       0.81      0.88      0.84       260
           1       0.73      0.61      0.66       138

    accuracy                           0.79       398
   macro avg       0.77      0.74      0.75       398
weighted avg       0.78      0.79      0.78       398

Classification Report for Train Set:
               precision    recall  f1-score   support

           0       0.85      0.94      0.89      1017
           1       0.86      0.72      0.78       572

    accuracy                           0.86      1589
   macro avg       0.86      0.83      0.84      1589
weighted avg       0.86      0.86      0.85      1589



KNeighborsClassifier(n_neighbors=3)